# Grad-CAM Example

In [1]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torchvision import transforms
import time
from copy import deepcopy

from models import MainModel
from models import GradCAM
from argparse import Namespace
from skimage.io import imsave, imread

In [2]:
model_densenet201 = MainModel('densenet201', 6)
model_densenet201.load_state_dict(torch.load('model_densenet201.pt'))
model_densenet201.eval()

# model_resnet101 = MainModel('resnet101', 6)
# model_resnet101.load_state_dict(torch.load('resnet101/model_resnet101.pt'))
# model_resnet101.eval()

# model_vgg19 = MainModel('vgg19', 6)
# model_vgg19.load_state_dict(torch.load('vgg19/model_vgg19.pt'))
# model_vgg19.eval()

print('')

In [23]:
ct_mean = 0.188
ct_std = 0.315
def transform(dirr):
    trans = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(degrees=10, translate=(0.02, 0.02)),
        transforms.ToTensor(),
        transforms.Normalize([ct_mean], [ct_std], inplace=True)
    ])
    return trans(
        Image.open(dirr)
        # One, a single image. Three, 3 channels. -1 keeps original.
        # (1,3,256.256)
    )

In [15]:
df = pd.read_csv('../label.csv')

# Which one's faster?

In [20]:
a = np.array(Image.open('../images/6d317b3d6.png'))
# a.shape
# transform(a)
transform(Image.fromarray(a))

tensor([[[-0.5968, -0.5968, -0.5968,  ..., -0.5968, -0.5968, -0.5968],
         [-0.5968, -0.5968, -0.5968,  ..., -0.5968, -0.5968, -0.5968],
         [-0.5968, -0.5968, -0.5968,  ..., -0.5968, -0.5968, -0.5968],
         ...,
         [-0.5968, -0.5968, -0.5968,  ..., -0.5968, -0.5968, -0.5968],
         [-0.5968, -0.5968, -0.5968,  ..., -0.5968, -0.5968, -0.5968],
         [-0.5968, -0.5968, -0.5968,  ..., -0.5968, -0.5968, -0.5968]]])

In [8]:
# images = []
# for i in df['ID'].values[:3]:
#     print(i)
#     images.append(transform(f'../images/{i}.png'))
trans = lambda e: (torch.tensor((np.array(Image.open(e))) - ct_mean )/ ct_std).expand(3,-1,-1)

t = time.time()
a = transform('../images/6d317b3d6.png')
print(time.time() - t)

t = time.time()
b = trans('../images/ab3c1fad5.png')
print(time.time() - t)

# c = transform('../images/df8281674.png')

# gc
# torch.stack((a,b,c)).shape

# images[0].shape

0.004407167434692383
0.0011792182922363281


# GRADCAM

In [162]:
transform('../images/ab3c1fad5.png').shape

torch.Size([1, 256, 256])

In [109]:
os.listdir('../images')[0]

'63547ebc5.png'

In [24]:
images = []
for img_file in os.listdir('../images'):
    if img_file == '.ipynb_checkpoints':
        continue
    img_file = f'../images/{img_file}'
    images.append(transform(img_file))
    

In [27]:
def jet(image):
    n = 4 * image[:, :1]
    r = torch.clamp(torch.min(n-1.5,-n+4.5), 0, 1)
    g = torch.clamp(torch.min(n-0.5,-n+3.5), 0, 1)
    b = torch.clamp(torch.min(n+0.5,-n+2.5), 0, 1)
    return torch.cat((r,g,b), 1)
def gc(model, image, ind):
    image = image.unsqueeze(0)
    ind = torch.tensor([[ind]])
    # Grad CAM
    grad_cam = GradCAM(model)
    cam = grad_cam(image, ind)
    # output image with cam
#     cam = jet(cam)
#     image = torch.clamp(image, 0, 1)
    image = image + cam
#     image = np.moveaxis(image[0].cpu().numpy(), 0, 2)
    image = image / image.max()
#     image = np.around(image*255).astype(np.uint8)
    
    return image[0].numpy()

In [40]:
im_ = gc(model_densenet201, images[10].expand(3,-1,-1), 0)[0]
# im_ = torch.clamp(torch.tensor(im_) * ct_std + ct_mean, 0, 1)
# imsave('2.png', np.array(np.around(im_*255)).astype(np.uint8))
# im_.shape
np.save('1.npy',im_)

In [39]:
np.load('1.npy').shape

(3, 256, 256)

In [25]:
for img in images[:1]:
    indexes = []
    indexes.append(img.numpy()[0])
    for ind in range(6):
        indexes.append(np.nan_to_num(gc(model_densenet201, img, ind)[0]))
    indexes = torch.tensor(indexes)

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 1, 256, 256] to have 3 channels, but got 1 channels instead

In [177]:
indexes.shape

torch.Size([7, 256, 256])

In [384]:
wa = transform('../images/6d317b3d6.png')
# agc = torch.tensor(gc(model_densenet201, a, 0))
# agc = torch.stack((agc[:,:,0]/255., agc[:,:,1]/255.,agc[:,:,2]/255.))
# print(torch.sigmoid(model_densenet201(agc.unsqueeze(0))))

In [314]:
df[df['ID'] == '6d317b3d6']

,ID,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,6d317b3d6,1,0,0,0,0,1


In [149]:
#       (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [348]:
# model_densenet201.feature.features

In [164]:
dn = deepcopy(model_densenet201)

In [152]:
dn =  torch.nn.Sequential(
    torch.nn.Conv2d(7, 3, (7,7), (2,2), (3,3), dilation=1, bias=False)
    ,model_densenet201)

In [147]:
a = indexes.unsqueeze(0)

In [169]:
dn.feature.features.conv0 = torch.nn.Sequential(
    torch.nn.Conv2d(7, 3, (7,7), (2,2), (3,3), dilation=1, bias=False),
    dn.feature.features.conv0
)